In [1]:
"""

Example input from file:
---
        ...#
        .#..
        #...
        ....
...#.......#
........#...
..#....#....
..........#.
        ...#....
        .....#..
        .#......
        ......#.

10R5L5R10L4R5L5
---

The first half of the monkeys' notes is a map of the board. It is comprised of a set of open tiles (on which you can move, drawn .) and solid walls (tiles which you cannot enter, drawn #).

The second half is a description of the path you must follow. It consists of alternating numbers and letters:

A number indicates the number of tiles to move in the direction you are facing. If you run into a wall, you stop moving forward and continue with the next instruction.
A letter indicates whether to turn 90 degrees clockwise (R) or counterclockwise (L). Turning happens in-place; it does not change your current tile.
So, a path like 10R5 means "go forward 10 tiles, then turn clockwise 90 degrees, then go forward 5 tiles".

You begin the path in the leftmost open tile of the top row of tiles. Initially, you are facing to the right (from the perspective of how the map is drawn).

If a movement instruction would take you off of the map, you wrap around to the other side of the board. In other words, if your next tile is off of the board, you should instead look in the direction opposite of your current facing as far as you can until you find the opposite edge of the board, then reappear there.

For example, if you are at A and facing to the right, the tile in front of you is marked B; if you are at C and facing down, the tile in front of you is marked D:

        ...#
        .#..
        #...
        ....
...#.D.....#
........#...
B.#....#...A
.....C....#.
        ...#....
        .....#..
        .#......
        ......#.

It is possible for the next tile (after wrapping around) to be a wall; this still counts as there being a wall in front of you, and so movement stops before you actually wrap to the other side of the board.

By drawing the last facing you had with an arrow on each tile you visit, the full path taken by the above example looks like this:

        >>v#    
        .#v.    
        #.v.    
        ..v.    
...#...v..v#    
>>>v...>#.>>    
..#v...#....    
...>>>>v..#.    
        ...#....
        .....#..
        .#......
        ......#.

To finish providing the password to this strange input device, you need to determine numbers for your final row, column, and facing as your final position appears from the perspective of the original map. Rows start from 1 at the top and count downward; columns start from 1 at the left and count rightward. (In the above example, row 1, column 1 refers to the empty space with no tile on it in the top-left corner.) Facing is 0 for right (>), 1 for down (v), 2 for left (<), and 3 for up (^). The final password is the sum of 1000 times the row, 4 times the column, and the facing.

In the above example, the final row is 6, the final column is 8, and the final facing is 0. So, the final password is 1000 * 6 + 4 * 8 + 0: 6032.

All indexing should be done with (row, col) tuples which are converted to (y, x) for numpy indexing

"""

import sys
import re
import numpy as np

In [28]:
from enum import Enum

# Direction enum
class Direction(Enum):
    RIGHT = 0
    DOWN = 1
    LEFT = 2
    UP = 3

    def symbols() -> list:
        return ['>', 'v', '<', '^']
    
    def tuples() -> list:
        return [(0, 1), (1, 0), (0, -1), (-1, 0)]

    def value_from(initial) -> 'Direction':
        if isinstance(initial, str):
            if initial == 'R':
                initial = 0
            elif initial == 'L':
                initial = 2
            else:
                initial = Direction.symbols().index(initial)
        elif isinstance(initial, tuple):
            initial = Direction.tuples().index(initial)
        elif isinstance(initial, Direction):
            initial = initial.value
        else:
            raise ValueError("Invalid direction: {}".format(initial))
        return Direction(initial)

    def turn(self, direction):
        direction = Direction.value_from(direction)
        if direction == Direction.RIGHT:
            return Direction((self.value + 1) % 4)
        elif direction == Direction.LEFT:
            return Direction((self.value - 1) % 4)
        else:
            raise ValueError("Invalid direction: {}".format(direction))

    def symbol(self):
        return Direction.symbols()[self.value]

    def tuple(self):
        return Direction.tuples()[self.value]

    def __str__(self):
        return self.symbol()

    def __repr__(self):
        return str(self)

assert Direction.RIGHT == Direction.value_from('>')
assert Direction.DOWN == Direction.value_from('v')
assert Direction.LEFT == Direction.value_from('<')
assert Direction.UP == Direction.value_from('^')
assert Direction.RIGHT == Direction.value_from(Direction.RIGHT)
assert Direction.DOWN == Direction.value_from(Direction.DOWN)
assert Direction.LEFT == Direction.value_from(Direction.LEFT)
assert Direction.UP == Direction.value_from(Direction.UP)
assert Direction.RIGHT == Direction.value_from((0, 1))
assert Direction.DOWN == Direction.value_from((1, 0))
assert Direction.LEFT == Direction.value_from((0, -1))
assert Direction.UP == Direction.value_from((-1, 0))
assert Direction.RIGHT == Direction.value_from(Direction.RIGHT.tuple())
assert Direction.DOWN == Direction.value_from(Direction.DOWN.tuple())
assert Direction.LEFT == Direction.value_from(Direction.LEFT.tuple())
assert Direction.UP == Direction.value_from(Direction.UP.tuple())

In [46]:
def parse_input(filename):
    """
    Reads a file (example below) and returns a 2d numpy array representing the board, and a list of instructions separated by number and letter. 

            ...#
            .#..
            #...
            ....
    ...#.......#
    ........#...
    ..#....#....
    ..........#.
            ...#....
            .....#..
            .#......
            ......#.

    10R5L5R10L4R5L5
    """
    board_lines = []
    instructions = []

    cube_map_lines = []
    transitions = {}
    with open(filename) as f:
        try:
            for line in f:
                if line.strip() == '':
                    break
                board_lines.append(line.rstrip('\n'))
            instructions = f.readline().rstrip('\n')
            instructions = re.findall(r'\d+|[RL]', instructions)
            f.readline()
            for line in f:
                if line.strip() == '':
                    break
                cube_map_lines.append(line.rstrip('\n'))
            for line in f:
                if line.strip() == '':
                    break
                matches = re.match(r'(\d) ([<>^v]) (\d) ([<>^v])', line)
                side1, side2 = int(matches[1]), int(matches[3])
                dir1, dir2 = Direction.value_from(matches[2]), Direction.value_from(matches[4])
                transitions[side1, dir1] = side2, dir2
                transitions[side2, dir2] = side1, dir1
        except Exception as e:
            print(line, e)
            sys.exit(1)
            
    line_len = max([len(line) for line in board_lines])
    # create an np array of len(board_lines) x line_len, with np.nan as the default value, and fill it with the board
    # print(board_lines)
    board = np.full((len(board_lines), line_len), ' ')
    for i, line in enumerate(board_lines):
        for j, char in enumerate(line):
            board[i,j] = char

    cube_map = board.copy()
    for i, line in enumerate(cube_map_lines):
        for j, char in enumerate(line):
            cube_map[i,j] = char
    
    return board, instructions, cube_map, transitions

board, instructions, cube_map, transitions = parse_input('./day22-input.txt')
# find the upper left and lower right corners of each numbered cube face in the cube map
cube_corners = {}
for i in range(1, 7):
    cube_corners[i] = np.where(cube_map == str(i))
    cube_corners[i] = (cube_corners[i][0][0], cube_corners[i][1][0]), (cube_corners[i][0][-1], cube_corners[i][1][-1])

In [ ]:
def print_board(board):
    for row in board:
        print(''.join(row))

print_board(board)
print_board(cube_map)
display(cube_corners)
display(transitions)

In [53]:

def move(t1, t2):
    return tuple([t1[i] + t2[i] for i in range(len(t1))])

# Returns the next position (X,Y) tuple in the given direction (X,Y) vector tuple from the pos (X,Y) tuple position on the numpy array game board.
def get_next_pos(pos, direction, board):
    # We can only walk on '.' characters. 
    # If we would be moving on to a ' ' character, we instead need to wrap around to the other side of the board to the first '.' character.
    # If we would be moving on to a '#' character, we need to stop.
    try:
        next_pos = move(pos, direction.tuple())

        # If the next position is a ' ' or off the board, we need to wrap around to the first non-blank character on the other side of the board
        if next_pos[0] < 0 or next_pos[1] < 0 or next_pos[0] >= board.shape[0] or next_pos[1] >= board.shape[1] or board[next_pos] == ' ':
            # find the first non-blank character in the direction we're moving, starting at the other side of the board
            if direction == Direction.RIGHT:
                # moving right, start at the left side of the board on this row
                next_pos = (pos[0], 0)
            elif direction == Direction.DOWN:
                # moving down, start at the top of the board on this column
                next_pos = (0, pos[1])
            elif direction == Direction.LEFT:
                # moving left, start at the right side of the board on this row
                next_pos = (pos[0], board.shape[1] - 1)
            elif direction == Direction.UP:
                # moving up, start at the bottom of the board on this column
                next_pos = (board.shape[0] - 1, pos[1])

            while board[next_pos] == ' ': # keep moving until we find a non-blank character
                next_pos = move(next_pos, direction.tuple())

        # If the next position is a '.', we're good to go
        if board[next_pos] == '.':
            return next_pos

        # If the next position is a '#', we need to stop
        if board[next_pos] == '#':
            return pos
    except IndexError as e:
        print("IndexError: pos={}, direction={}, next_pos={}, board.shape={}".format(pos, direction, next_pos, board.shape))
        raise e

# Move local row/column in current face to local row/column in next face
# Rules: 
# 1) If the current direction is pointing up and the entry side is ^, the new row is 0 and the column is the max column - current column
# 2) If the current direction is pointing down and the entry side is v, the new row is the max row and the column is the max column - current column
# 3) If the current direction is pointing left and the entry side is <, the new column is 0 0 and the row is the max row - current row
# 4) If the current direction is pointing right right and the entry side is >, the new column is the max column and the row is the max row - current row
# 5) If the current direction is pointing left and the entry side is ^, the new column is the current row and the new row is 0
# 6) If the current direction is pointing left and the entry side is v, the new column is the current row and the new row is the max row
# 7) If the current direction is pointing right right and the entry side is ^, the new column is the max column - the current row and the new row is 0
# 8) If the current direction is pointing right right and the entry side is v, the new column is the max column - the current row and the new row is the max row
# 9) If the current direction is pointing up and the entry side is <, the new row is the current column and the new column is 0
# 10) If the current direction is pointing up and the entry side is >, the new row is the current column and the new column is the max column
# 11) If the current direction is pointing down and the entry side is <, the new row is the max row - the current column and the new column is 0
# 12) If the current direction is pointing down and the entry side is >, the new row is the max row - the current column and the new column is the max column
def get_next_side_pos_dir(curr_side_pos, curr_direction, next_side, next_side_entry):
    max_row = cube_corners[next_side][1][0] - cube_corners[next_side][0][0]
    max_col = cube_corners[next_side][1][1] - cube_corners[next_side][0][1]

    if curr_direction == Direction.UP:
        if next_side_entry == Direction.UP:
            next_pos = (0, max_col - curr_side_pos[1])
        elif next_side_entry == Direction.LEFT:
            next_pos = (curr_side_pos[1], 0)
        elif next_side_entry == Direction.RIGHT:
            next_pos = (max_row - curr_side_pos[1], max_col)
        elif next_side_entry == Direction.DOWN:
            next_pos = (max_row, curr_side_pos[1])
    elif curr_direction == Direction.DOWN:
        if next_side_entry == Direction.DOWN:
            next_pos = (max_row, max_col - curr_side_pos[1])
        elif next_side_entry == Direction.LEFT:
            next_pos = (max_row - curr_side_pos[1], 0)
        elif next_side_entry == Direction.RIGHT:
            next_pos = (curr_side_pos[1], max_col)
        elif next_side_entry == Direction.UP:
            next_pos = (0, curr_side_pos[1])
    elif curr_direction == Direction.LEFT:
        if next_side_entry == Direction.LEFT:
            next_pos = (max_row - curr_side_pos[0], 0)
        elif next_side_entry == Direction.UP:
            next_pos = (0, curr_side_pos[0])
        elif next_side_entry == Direction.DOWN:
            next_pos = (max_row, max_col - curr_side_pos[0])
        elif next_side_entry == Direction.RIGHT:
            print("not in current data")
            next_pos = (max_row - curr_side_pos[0], max_col)
    elif curr_direction == Direction.RIGHT:
        if next_side_entry == Direction.RIGHT:
            next_pos = (max_row - curr_side_pos[0], max_col)
        elif next_side_entry == Direction.UP:
            next_pos = (0, max_row - curr_side_pos[0])
        elif next_side_entry == Direction.DOWN:
            next_pos = (max_row, curr_side_pos[0])
        elif next_side_entry == Direction.LEFT:
            print("not in current data")
            next_pos = (max_row - curr_side_pos[0], 0)            
    
    next_dir = next_side_entry.turn('R').turn('R')
    
    return next_pos, next_dir

# Returns the next position (X,Y) tuple in the given direction (X,Y) vector tuple from the pos (X,Y) tuple position on the numpy array game board.
def get_next_pos_cube(pos, direction, board, cube_map, transitions):
    # If we would be moving on to a ' ' character or off the edge of the board, we instead need to wrap around to the first '.' character in the adjoining side of the cube map.    
    # We can only walk on '.' characters. 
    # If we would be moving on to a '#' character, we need to stop.
    try:
        next_pos = move(pos, direction.tuple())
        next_direction = direction

        # If the next position is a ' ' or off the board, we need to wrap around to the first non-blank character in the adjoining side of the cube map.  
        if next_pos[0] < 0 or next_pos[1] < 0 or next_pos[0] >= board.shape[0] or next_pos[1] >= board.shape[1] or board[next_pos] == ' ':
            # curr_cube_side
            curr_side = int(cube_map[pos])
            curr_local_pos = (pos[0] - cube_corners[curr_side][0][0], pos[1] - cube_corners[curr_side][0][1])
            next_side, next_side_entry = transitions[curr_side, direction]
            
            try:
                next_local_pos, next_direction = get_next_side_pos_dir(curr_local_pos, direction, next_side, next_side_entry)
                # print("Going {} from side {} at {}, entering side {} on {} at {} heading {}".format(direction, curr_side, curr_local_pos, next_side, next_side_entry, next_local_pos, next_direction))
                next_pos = (next_local_pos[0] + cube_corners[next_side][0][0], next_local_pos[1] + cube_corners[next_side][0][1])
            except Exception as e:
                print("curr_side={}, curr_local_pos={}, curr_dir_symbol={}, next_side={}, next_side_entry={}".format(curr_side, curr_local_pos, direction, next_side, next_side_entry))
                raise e
                
            while board[next_pos] == ' ': # keep moving until we find a non-blank character
                next_pos = move(next_pos, next_direction)

        # If the next position is a '.', we're good to go
        if board[next_pos] == '.':
            return next_pos, next_direction

        # If the next position is a '#', we need to stop
        if board[next_pos] == '#':
            return pos, direction
    except IndexError as e:
        print("IndexError: pos={}, direction={}, next_pos={}, board.shape={}".format(pos, direction, next_pos, board.shape))
        raise e

In [ ]:
# find the first 0 value in the first row
start_col = np.where(board[0,:] == '.')[0][0]

pos = (0, start_col)
direction = Direction.RIGHT

path = board.copy()
steps = 0
sub_steps = 0
try:
    for command in instructions:
        # print(command)
        path[pos] = direction.symbol()
        steps += 1
        if command in ['R', 'L']:
            old_dir = direction
            direction = direction.turn(command)
        else:
            sub_steps = 0
            for i in range(int(command)):
                sub_steps += 1 
                old_pos = pos
                old_dir = direction
                # pos = get_next_pos(pos, direction, board)
                pos, direction = get_next_pos_cube(pos, direction, board, cube_map, transitions)
                if pos == old_pos:
                    # print("Stopped at {} in direction {} on board shape {}".format(pos, direction, board.shape))
                    break
                
                # print("Moved from {} in direction {} to {} on board shape {}".format(old_pos, direction, pos, board.shape))
                path[pos] = direction.symbol()
            sub_steps = 0
except Exception as e:
    print("Exception at step {} ({}), command {}: {}".format(steps, sub_steps, command, e))
    raise e
    

final_row, final_col = move(pos, (1,1)) # offset for 0s
print("Final row: {}, final col: {}, final heading: {}".format(final_row, final_col, direction))
print("Final password: {}".format((1000 * final_row) + (4 * final_col) + direction.value))


In [ ]:
print_board(path)